# Data Formatting

## Téléchargement des données de la SRTM

Il nous a fallu dans un premier temps télécharger les données cartographiques de la SRTM, l'interface de leur site n'étant pas adapté à un téléchargement massif de données, nous avons développé un script permettant d'automatiser les téléchargements. 

In [ ]:
import requests
import os

for i in range(1, 72):
    for j in range(1, 24):
        if os.path.isfile("../downloaded/srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2))):
            print("[Found] srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))
        else:
            response = requests.get("http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))
            if response.status_code == 200:
                file_name = "srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2))
                file_path = "../downloaded/" + file_name
                with open(file_path, 'wb') as f:
                    print("[Downloaded]: " + file_name)
                    f.write(response.content)
            else:
                print("[Not Found] srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))

Une fois ces données en notre possession, il a fallu créer un extracteur pour récupérer uniquement nos fichiers .tif des archives téléchargées

In [ ]:
import os
import fnmatch
import zipfile


def extractor(zip_file):
    extensions = '.tif'
    [zip_file.extract(file, r"../downloaded/tif") for file in zip_file.namelist() if file.endswith(extensions)]


def extractTifFiles():
    if not os.path.exists("../downloaded/tif"):
        os.makedirs("../downloaded/tif")
    pattern = '*.zip'
    for root, dirs, files in os.walk('../downloaded'):
        for filename in fnmatch.filter(files, pattern):
            if os.path.isfile(os.path.join('../downloaded/tif/', os.path.splitext(filename)[0] + '.tif')):
                print(os.path.splitext(filename)[0] + '.tif trouvé')
                continue
            with zipfile.ZipFile(os.path.join(root, filename)) as zf:
                extractor(zf)
                print(filename, "extrait!")


if __name__ == "__main__":
    extractTifFiles()

Enfin, après avoir extrait nos images, il convenait de les réduire de manière à pouvoir les utiliser dans notre classificateur, nous avons donc subdivisé nos images d'une taille de 6000x6000px en 100 images plus petites d'une taille de 600x600px à l'aide de la librairie GDAL permettant de faire des opérations sur des GeoTIFF.

In [ ]:
import os
from osgeo import gdal


def split(path):
    raw_file_name = os.path.splitext(os.path.basename(path))[0].replace("_downsample", "")

    tile_size_x = 600
    tile_size_y = 600

    ds = gdal.Open(path)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize

    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i) + ", " + str(j) + ", " \
                         + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(path) \
                         + " " + str("../downloaded/subdivided/") + str(raw_file_name) + "_" \
                         + str(i) + "_" + str(j) + ".tif"
            os.system(com_string)


if __name__ == "__main__":
    for file in os.listdir('../downloaded/tif'):
        if file.endswith(".tif"):
            split(os.path.join('../downloaded/tif', file))

Nous avons aussi développé une version permettant de les convertir directement en JPEG, les données GeoTiff n'étant pas nécessaire pour notre utilisation.

In [ ]:
from PIL import Image
import os


def split(file, chopsize, basename):
    img = Image.open(file)
    width, height = img.size
    for x0 in range(0, width, chopsize):
        for y0 in range(0, height, chopsize):
            box = (x0, y0,
                   x0 + chopsize if x0 + chopsize < width else width - 1,
                   y0 + chopsize if y0 + chopsize < height else height - 1)
            print('%s %s' % (file, box))
            img.crop(box).convert('RGB').save('../downloaded/subdivided/%s_x%03d_y%03d.jpg' % (basename, x0, y0))


if __name__ == "__main__":
    for file in os.listdir('../downloaded/tif'):
        basename = os.path.basename(file).replace('.tif', '')
        if file.endswith(".tif"):
            split(os.path.join('../downloaded/tif/', file), 600, basename)


## Téléchargement d'images de "non terrains"

Afin de pouvoir réaliser notre classificateur, nous avions besoin de données n'étant pas des terrains. C'est ainsi que le développement d'un script de téléchargement d'images aléatoires nous est apparu nécessaire.

Notre script est relié à une API nous fournissant une image aléatoire de la même taille que nos images de la SRTM utilisées dans notre classificateur (600x600px). Il l'utilise afin de télécharger des images pour l'entraînement, la validation ainsi que pour les tests.

In [ ]:
import requests

# Training
for i in range(200):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/training/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Validation
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/validation/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Test
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/test/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)